# Train CNN CIFAR10 classifier with Keras

In [12]:
!pip install matplotlib seaborn pandas numpy scipy

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [13]:
!pwd

/root/keras/tensorflow/keras_experiments/cifar10


In [14]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution is: {}".format(tf.executing_eagerly()))
print("Keras version: {}".format(tf.keras.__version__))

TensorFlow version: 2.1.0
Eager execution is: True
Keras version: 2.2.4-tf


# CIFAR-10
The [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.



class_names = \[
    'airplane', 
    'automobile', 
    'bird', 
    'cat', 
    'deer', 
    'dog', 
    'frog', 
    'horse', 
    'ship', 
    'truck'
\]

<img src="images/cifar10.png"
     alt="Markdown Monster icon"
     style="float: left; margin-right: 10px;" 
     width=850/>

## CIFAR10 Utilities

In [15]:
IMAGE_WIDTH = 32
IMAGE_HEIGHT = 32
NUM_CHANNELS = 3

In [16]:
def plot_cifar10_image(image, width, height, channels):
    reshaped = image.reshape(width, height, channels)
    plt.figure(figsize=(1,1)),
    plt.imshow(reshaped, cmap=plt.cm.binary)
    plt.axis("off")
    
def plot_cifar10_images(instances, width=IMAGE_WIDTH, height=IMAGE_HEIGHT, channels=NUM_CHANNELS, images_per_row=5, **options):
    plt.figure(figsize=(6,6))

    images_per_row = min(len(instances), images_per_row)
    images = [instance.reshape(width, height, channels) for instance in instances]
    n_rows = (len(instances) - 1) // images_per_row + 1
    row_images = []
    n_empty = n_rows * images_per_row - len(instances)
    images.append(np.zeros((width, width * n_empty)))
    for row in range(n_rows):
        rimages = images[row * images_per_row : (row + 1) * images_per_row]
        row_images.append(np.concatenate(rimages, axis=1))
    image = np.concatenate(row_images, axis=0)
    plt.imshow(image, cmap = mpl.cm.binary, **options)
    plt.axis("off")

## Load data

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

### Shape

In [ ]:
# Reshape data based on channels first / channels last strategy.
# This is dependent on whether you use TF, Theano or CNTK as backend.
# Source: https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py
if K.image_data_format() == 'channels_first':
    input_shape = (1, IMAGE_WIDTH, IMAGE_HEIGHT)
else:
    input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CHANNELS,)

### Label classes

In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist]
NUMBER_OF_CLASSES = len(set(flatten(y_train)))
NUMBER_OF_CLASSES = 10
print("NUMBER_OF_CLASSES", NUMBER_OF_CLASSES)

### Sample images

In [ ]:
%matplotlib inline
plot_cifar10_image(x_train[0], IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CHANNELS)
plt.show()

In [ ]:
plot_cifar10_images(x_train[0:25], IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CHANNELS)

## Shuffle data

In [ ]:
# Permutation/Shuffle
indexes = np.arange(x_train.shape[0])
indexes = np.random.permutation(indexes)  # shuffle data to randomly select
x_train = x_train[indexes]
y_train = y_train[indexes] 

## Normalize data

In [ ]:
# Using Keras normalization
#x_full = np.concatenate((x_train, x_test), axis=0)
#normalization = Normalization()
#normalization.adapt(x_full)
#x_train = normalization(x_train / 1.0).numpy()
#x_test = normalization(x_test / 1.0).numpy()

   

# Cast
#x_train, x_test = tf.cast(x_train, tf.float32), tf.cast(x_test, tf.float32)
#y_train, y_test = tf.cast(y_train, tf.uint8), tf.cast(y_test, tf.uint8)

x_train = x_train / 255.0
x_test = x_test / 255.0

## Allocate validation data

In [ ]:
x_train, x_valid = x_train[:40000], x_train[40000:]
y_train, y_valid = y_train[:40000], y_train[40000:]

print(x_train[0][1])
print(x_train[0].shape)
print(x_train.shape)
print(x_train.dtype)
print(y_train.shape)
print(y_train.dtype)

### Data argumentation
Shift, zoom, rotate, shear, flip the images

In [ ]:
"""
# DO NOT normalize/standarlize data itself, or need to do the same to the test/validation data
generator = ImageDataGenerator(
    featurewise_center=True,              # Set input mean to 0 over the dataset, feature-wise. 
    featurewise_std_normalization=True,   # Divide inputs by std of the dataset, feature-wise.
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
"""
generator = ImageDataGenerator(
    width_shift_range=0.1,  # fraction of total width if < 1 or pixels if >= 1
    height_shift_range=0.1, # fraction of total height if < 1 or pixels if >= 1
    shear_range=5,          # displaces each point in fixed direction with shear angle in counter-clockwise direction in degrees
    zoom_range=(0.9, 1.1),  # single float zoom_range ([1-zoom_range, 1+zoom_range]) or [lower, upper].
    rotation_range=10        # Int. degree range for random rotations.
    horizontal_flip=False,
    vertical_flip=True, 
    fill_mode='nearest',   # Points outside the boundaries of the input are filled according to the given mode. {"constant", "nearest", "reflect" or "wrap"}
    # cval=0                 # Float or Int. Value used for points outside the boundaries when fill_mode = "constant".
)
# Not necessary as no normalization/standadization.
# generator.fit(x_train)

# Training
* C: Convolution layer
* P: Pooling layer
* B: Batch normalization layer
* F: Fully connected layer
* O: Output fully connected softmax layer

## Hyper parameters

In [ ]:
BATCH_SIZE = 32
NUMBER_OF_EPOCHS = 200
validation_split = 0.2
verbosity = 1
use_multiprocessing = True
workers = 4

DROPOUT_RATE = 0.3

## Callbacks

In [ ]:
# tensorboard --logdir=/full_path_to_your_logs

import os
log_dir = os.getcwd() + os.path.sep + "logs"
print(log_dir)

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    write_graph=True,
    write_images=True,
    histogram_freq=1,  # How often to log histogram visualizations
    embeddings_freq=1,  # How often to log embedding visualizations
    update_freq="epoch",
)  # How often to write logs (default: once per epoch)

### Early stop callback

In [ ]:
earlystop_callback = tf.keras.callbacks.EarlyStopping(
    #monitor='accuracy',
    #monitor='val_loss',
    monitor='val_accuracy',
    min_delta=0,
    mode='auto',
    patience=10,
    restore_best_weights=True
)

## Model Traing

### Kernel regularizer

In [ ]:
l2_loss_lambda = 0.010
kernel_regularizer = regularizers.l2(l2_loss_lambda)

### Trainer

In [ ]:
def train(model, x, y):
    model.compile(
        optimizer='adam', 
        loss=tf.keras.losses.sparse_categorical_crossentropy, 
        metrics=['accuracy']
    )
    history = model.fit(
        generator.flow(x, y, batch_size=BATCH_SIZE),
        steps_per_epoch=len(x_train) / BATCH_SIZE, 
        epochs=NUMBER_OF_EPOCHS,
        shuffle=True,
        validation_data=(x_valid, y_valid),
        use_multiprocessing=use_multiprocessing,
        workers=workers,
        verbose=verbosity,
        callbacks=[
#            tensorboard_callback,
            earlystop_callback
        ]
    )
    return history

### Train CPBFO model  (F/ReLU/He)

In [ ]:
model = Sequential([
    #normalization,
    BatchNormalization(
        name="input_normalization"
    ),
    Conv2D(
        name="conv01",
        filters=32, 
        kernel_size=(3, 3), 
        strides=(1, 1), 
        padding="same",
        activation='relu', 
        input_shape=input_shape
    ),
    MaxPooling2D(
        name="pool01",
        pool_size=(2, 2)
    ),
    Flatten(),                                        # 3D shape to 1D.
    BatchNormalization(
        name="batch_before_full01"
    ),
    Dense(
        name="full01", 
        units=300, 
        activation="relu"
    ),     # Fully connected layer 
    Dense(
        name="output_softmax", 
        units=NUMBER_OF_CLASSES, 
        activation="softmax"
    )
])

history = train(model, x_train, y_train)
print(history)
results = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=1)
print("test loss, test accuracy:", results)

In [ ]:
model = Sequential([
    #normalization,
    BatchNormalization(
        name="input_normalization"
    ),
    Conv2D(
        name="conv01",
        filters=32, 
        kernel_size=(3, 3), 
        strides=(1, 1), 
        padding="same",
        activation='relu', 
        input_shape=input_shape
    ),
    MaxPooling2D(
        name="pool01",
        pool_size=(2, 2)
    ),
    Flatten(),                                        # 3D shape to 1D.
    BatchNormalization(
        name="batch_before_full01"
    ),
    Dense(
        name="full01", 
        units=300, 
        activation="relu"
    ),     # Fully connected layer 
    Dense(
        name="output_softmax", 
        units=NUMBER_OF_CLASSES, 
        activation="softmax"
    )
])

history = train(model, x_train, y_train)
print(history)
results = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=1)
print("test loss, test accuracy:", results)

### Train CPBCPBFO model (F/ReLU/He)

In [ ]:
model = Sequential([
#    normalization,
    BatchNormalization(
        name="input_normalization"
    ),
    Conv2D(
        name="conv_01",
        filters=32, 
        kernel_size=(3, 3), 
        strides=(1, 1), 
        padding="same",
        activation='relu', 
        input_shape=input_shape
    ),
    MaxPooling2D(                                     
        pool_size=(2, 2)
    ),
    BatchNormalization(),
    Conv2D(                                           
        filters=64, 
        kernel_size=(3, 3), 
        strides=(1, 1), 
        padding="same",
        activation='relu', 
        input_shape=input_shape
    ),
    MaxPooling2D(                                     
        pool_size=(2, 2)
    ),
    Flatten(),                                        # 3D shape to 1D.
    BatchNormalization(),
    Dense(300, activation="relu"),                    # Fully connected layer 
    Dense(NUMBER_OF_CLASSES, activation="softmax")
])
model.layers

train(model, x_train, y_train)
results = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE)
print("test loss, test accuracy:", results)

### Train CPBCPBFBFO model (F/ReLU/He)

In [ ]:
model = Sequential([
#    normalization,
    BatchNormalization(
        name="input_normalization"
    ),
    Conv2D(                                           
        filters=32, 
        kernel_size=(3, 3), 
        strides=(1, 1), 
        padding="same",
        activation='relu', 
        input_shape=input_shape
    ),
    MaxPooling2D(                                     
        pool_size=(2, 2)
    ),
    BatchNormalization(),
    Conv2D(                                           
        filters=64, 
        kernel_size=(3, 3), 
        strides=(1, 1), 
        padding="same",
        activation='relu', 
        input_shape=input_shape
    ),
    MaxPooling2D(                                     
        pool_size=(2, 2)
    ),
    Flatten(),                                        # 3D shape to 1D.
    BatchNormalization(),
    Dense(300, activation="relu"),                    # Fully connected layer 
    BatchNormalization(),
    Dense(200, activation="relu"),                    # Fully connected layer 
    Dense(NUMBER_OF_CLASSES, activation="softmax")    # Output layer
])

train(model, x_train, y_train)
results = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE)
print("test loss, test accuracy:", results)

### Train CPBCPBCPBFBFO model (F/ReLU/He)

In [ ]:
model = Sequential([
#    normalization,
    BatchNormalization(
        name="input_normalization"
    ),
    Conv2D(                                           
        filters=32, 
        kernel_size=(3, 3), 
        strides=(1, 1), 
        padding="same",
        activation='relu', 
        input_shape=input_shape
    ),
    MaxPooling2D(                                     
        pool_size=(2, 2)
    ),
    BatchNormalization(),
    Conv2D(                                           
        filters=64, 
        kernel_size=(3, 3), 
        strides=(1, 1), 
        padding="same",
        activation='relu', 
        input_shape=input_shape
    ),
    MaxPooling2D(                                     
        pool_size=(2, 2)
    ),
    BatchNormalization(),
    Conv2D(                                           
        filters=128, 
        kernel_size=(3, 3), 
        strides=(1, 1), 
        padding="same",
        activation='relu', 
        input_shape=input_shape
    ),
    MaxPooling2D(                                     
        pool_size=(2, 2)
    ),
    Flatten(),                                        # 3D shape to 1D.
    BatchNormalization(),
    Dense(300, activation="relu"),                    # Fully connected layer 
    BatchNormalization(),
    Dense(200, activation="relu"),                    # Fully connected layer 
    Dense(NUMBER_OF_CLASSES, activation="softmax")    # Output layer
])

train(model, x_train, y_train)
results = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE)
print("test loss, test accuracy:", results)

In [ ]:
model = Sequential([
    BatchNormalization(
        name="input_normalization"
    ),
    Conv2D(                                           
        filters=32, 
        kernel_size=(3, 3), 
        strides=(1, 1), 
        padding="same",
        activation='relu', 
        input_shape=input_shape
    ),
    MaxPooling2D(                                     
        pool_size=(2, 2)
    ),
    BatchNormalization(),
    Conv2D(                                           
        filters=64, 
        kernel_size=(3, 3), 
        strides=(1, 1), 
        padding="same",
        activation='relu', 
        input_shape=input_shape
    ),
    MaxPooling2D(                                     
        pool_size=(2, 2)
    ),
    Dropout(DROPOUT_RATE),
    
    BatchNormalization(),
    Conv2D(                                           
        filters=128, 
        kernel_size=(3, 3), 
        strides=(1, 1), 
        padding="same",
        activation='relu', 
        input_shape=input_shape
    ),
    MaxPooling2D(                                     
        pool_size=(2, 2)
    ),
    Dropout(DROPOUT_RATE),
    Flatten(),                                        # 3D shape to 1D.

    BatchNormalization(),
    Dense(300, activation="relu"),                    # Fully connected layer 
    Dropout(DROPOUT_RATE),

    BatchNormalization(),
    Dense(200, activation="relu"),                    # Fully connected layer 
    Dense(NUMBER_OF_CLASSES, activation="softmax")    # Output layer
])

train(model, x_train, y_train)
results = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE)
print("test loss, test accuracy:", results)

## Train NCPBCP(BC)+P(BF)+O model (F/ReLU/He)

In [ ]:
model = Sequential([
#    normalization,
    BatchNormalization(
        name="input_normalization"
    ),
    Conv2D(                                           
        filters=32, 
        kernel_size=(3, 3), 
        kernel_regularizer=kernel_regularizer,
        strides=(1, 1), 
        padding="same",
        activation='relu', 
        input_shape=input_shape
    ),
    MaxPooling2D(                                     
        pool_size=(2, 2)
    ),
    BatchNormalization(),
    Conv2D(                                           
        filters=64, 
        kernel_size=(3, 3), 
        kernel_regularizer=kernel_regularizer,
        strides=(1, 1), 
        padding="same",
        activation='relu', 
        input_shape=input_shape
    ),
    MaxPooling2D(                                     
        pool_size=(2, 2)
    ),
    BatchNormalization(),
    Conv2D(                                           
        filters=128, 
        kernel_size=(3, 3), 
        kernel_regularizer=kernel_regularizer,
        strides=(1, 1), 
        padding="same",
        activation='relu', 
        input_shape=input_shape,
    ),
    BatchNormalization(),
    Conv2D(                                           
        filters=128, 
        kernel_size=(3, 3), 
        kernel_regularizer=kernel_regularizer,
        strides=(1, 1), 
        padding="same",
        activation='relu', 
        input_shape=input_shape

    ),
    BatchNormalization(),
    Conv2D(                                           
        filters=128, 
        kernel_size=(3, 3), 
        kernel_regularizer=kernel_regularizer,
        strides=(1, 1), 
        padding="same",
        activation='relu', 
        input_shape=input_shape
    ),
    BatchNormalization(),
    Conv2D(                                           
        filters=128, 
        kernel_size=(3, 3), 
        kernel_regularizer=kernel_regularizer,
        strides=(1, 1), 
        padding="same",
        activation='relu', 
        input_shape=input_shape
    ),
    BatchNormalization(),
    Conv2D(                                           
        filters=128, 
        kernel_size=(3, 3), 
        kernel_regularizer=kernel_regularizer,
        strides=(1, 1), 
        padding="same",
        activation='relu', 
        input_shape=input_shape
    ),
    BatchNormalization(),
    Conv2D(                                           
        filters=128, 
        kernel_size=(3, 3), 
        kernel_regularizer=kernel_regularizer,
        strides=(1, 1), 
        padding="same",
        activation='relu', 
        input_shape=input_shape
    ),
    MaxPooling2D(                                     
        pool_size=(2, 2)
    ),
    Flatten(),                                        # 3D shape to 1D.
    BatchNormalization(),
    Dense(
        units=300, 
        kernel_regularizer=kernel_regularizer,
        activation="relu"
    ),                    # Fully connected layer 
    BatchNormalization(),
    Dense(
        units=200, 
        kernel_regularizer=kernel_regularizer,
        activation="relu"
    ),                    # Fully connected layer 
    Dense(NUMBER_OF_CLASSES, activation="softmax")    # Output layer
])

train(model, x_train, y_train)
results = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE)
print("test loss, test accuracy:", results)

## Train NCPBCP(BC)+P(BF)+O model (F/ReLU/He) + Drop outs

In [ ]:
model = Sequential([
#    normalization,
    BatchNormalization(
        name="input_normalization"
    ),
    Conv2D(                                           
        filters=32, 
        kernel_size=(3, 3), 
        kernel_regularizer=kernel_regularizer,
        strides=(1, 1), 
        padding="same",
        activation='relu', 
        input_shape=input_shape
    ),
    MaxPooling2D(                                     
        pool_size=(2, 2)
    ),
    BatchNormalization(),
    Conv2D(                                           
        filters=64, 
        kernel_size=(3, 3), 
        kernel_regularizer=kernel_regularizer,
        strides=(1, 1), 
        padding="same",
        activation='relu', 
        input_shape=input_shape
    ),
    MaxPooling2D(                                     
        pool_size=(2, 2)
    ),
    Dropout(DROPOUT_RATE),
    
    BatchNormalization(),
    Conv2D(                                           
        filters=128, 
        kernel_size=(3, 3), 
        kernel_regularizer=kernel_regularizer,
        strides=(1, 1), 
        padding="same",
        activation='relu', 
        input_shape=input_shape,
    ),
    BatchNormalization(),
    Conv2D(                                           
        filters=128, 
        kernel_size=(3, 3), 
        kernel_regularizer=kernel_regularizer,
        strides=(1, 1), 
        padding="same",
        activation='relu', 
        input_shape=input_shape

    ),
    Dropout(DROPOUT_RATE),
    
    BatchNormalization(),
    Conv2D(                                           
        filters=128, 
        kernel_size=(3, 3), 
        kernel_regularizer=kernel_regularizer,
        strides=(1, 1), 
        padding="same",
        activation='relu', 
        input_shape=input_shape
    ),
    BatchNormalization(),
    Conv2D(                                           
        filters=128, 
        kernel_size=(3, 3), 
        kernel_regularizer=kernel_regularizer,
        strides=(1, 1), 
        padding="same",
        activation='relu', 
        input_shape=input_shape
    ),
    BatchNormalization(),
    Conv2D(                                           
        filters=128, 
        kernel_size=(3, 3), 
        kernel_regularizer=kernel_regularizer,
        strides=(1, 1), 
        padding="same",
        activation='relu', 
        input_shape=input_shape
    ),
    BatchNormalization(),
    Conv2D(                                           
        filters=128, 
        kernel_size=(3, 3), 
        kernel_regularizer=kernel_regularizer,
        strides=(1, 1), 
        padding="same",
        activation='relu', 
        input_shape=input_shape
    ),
    MaxPooling2D(                                     
        pool_size=(2, 2)
    ),
    Dropout(DROPOUT_RATE),
    
    Flatten(),                                        # 3D shape to 1D.
    BatchNormalization(),
    Dense(
        units=300, 
        kernel_regularizer=kernel_regularizer,
        activation="relu"
    ),                    # Fully connected layer 
    Dropout(DROPOUT_RATE),
    
    BatchNormalization(),
    Dense(
        units=200, 
        kernel_regularizer=kernel_regularizer,
        activation="relu"
    ),                    # Fully connected layer 
    
    Dense(NUMBER_OF_CLASSES, activation="softmax")    # Output layer
])

train(model, x_train, y_train)
results = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE)
print("test loss, test accuracy:", results)

### Train CPBFO model  (F/SELU/LeCun)
SELU activation + LeCun initialization at Fully connected layer

In [ ]:
model = Sequential([
    normalization,
    Conv2D(                                           
        filters=32, 
        kernel_size=(3, 3), 
        strides=(1, 1), 
        padding="same",
        kernel_initializer='he_normal', 
        activation='relu',
        input_shape=input_shape
    ),
    MaxPooling2D(                                     
        pool_size=(2, 2)
    ),
    Flatten(),                                        # 3D shape to 1D.
    BatchNormalization(),
    Dense(
        300, 
        kernel_initializer='lecun_normal', 
        activation='selu'
    ),                    # Fully connected layer 
    Dense(
        NUMBER_OF_CLASSES, 
        activation="softmax"
    )
])

train(model, x_train, y_train)
results = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE)
print("test loss, test accuracy:", results)

### Train CPBCPBFO model (F/SELU/LeCun)
SELU activation + LeCun initialization at Fully connected layer

In [ ]:
model = Sequential([
    normalization,
    Conv2D(                                           
        filters=32, 
        kernel_size=(3, 3), 
        strides=(1, 1), 
        padding="same",
        kernel_initializer='he_normal', 
        activation='relu',
        input_shape=input_shape
    ),
    MaxPooling2D(                                     
        pool_size=(2, 2)
    ),
    BatchNormalization(),
    Conv2D(                                           
        filters=64, 
        kernel_size=(3, 3), 
        strides=(1, 1), 
        padding="same",
        kernel_initializer='he_normal', 
        activation='relu',
        input_shape=input_shape
    ),
    MaxPooling2D(                                     
        pool_size=(2, 2)
    ),
    Flatten(),                                        # 3D shape to 1D.
    BatchNormalization(),
    Dense(
        300, 
        kernel_initializer='lecun_normal', 
        activation='selu'
    ),                    # Fully connected layer 
    Dense(
        NUMBER_OF_CLASSES, 
        activation="softmax"
    )
])

train(model, x_train, y_train)
results = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE)
print("test loss, test accuracy:", results)

### Train CPBCPBFFO model  (F/SELU/LeCun)
SELU activation + LeCun initialization at Fully connected layer

In [ ]:
model = Sequential([
    normalization,
    Conv2D(                                           
        filters=32, 
        kernel_size=(3, 3), 
        strides=(1, 1), 
        padding="same",
        kernel_initializer='he_normal', # Using SELU+LeCun damages the validation
        activation='relu',
        input_shape=input_shape
    ),
    MaxPooling2D(                                     
        pool_size=(2, 2)
    ),
    BatchNormalization(),
    Conv2D(                                           
        filters=64, 
        kernel_size=(3, 3), 
        strides=(1, 1), 
        padding="same",
        kernel_initializer='he_normal', 
        activation='relu',
        input_shape=input_shape
    ),
    MaxPooling2D(                                     
        pool_size=(2, 2)
    ),
    Flatten(),                                        # 3D shape to 1D.
    BatchNormalization(),
    Dense(
        300, 
        kernel_initializer='lecun_normal', 
        activation='selu'
    ),                    # Fully connected layer 
    Dense(
        200, 
        kernel_initializer='lecun_normal', 
        activation='selu'
    ),                    # Fully connected layer 
    Dense(
        NUMBER_OF_CLASSES, 
        activation="softmax"
    )
])

train(model, x_train, y_train)
results = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE)
print("test loss, test accuracy:", results)